<b>Dans ce notebook on ressort des éléments permettant de prendre connaissance du dataset et d'en déduire les actions de clean a effectuer</b>
<p><a href="https://app.mindmup.com/map/_free/2021/01/175c91405bdb11eb9f33cf002549c867">mind map du projet</a></p>

# Table of contents
1. [Load du dataset](#load)
2. [Examen de forme](#exam)
3. [Définition des mots clés](#keys)

   3.1 [Catégories de produit](#categ)
   
   3.2 [Groupes de produit](#groups)
4. [Analyse quantitative des features](#qanalyse)

   4.1 [Full NaN features](#full_nan)
   
   4.2 [Zero NaN features](#zero_nan)
   
   4.3 [Partial NaN features](#partial_nan)
   
5. [Audit du dataset](#actors)

   5.1 [Qui sont les créateurs d'information](#creators)
   
   5.2 [Quels pays sont représentés](#countries)
   
   5.3 [Quelles marques sont représentées](#brands)
   
   5.4 [Quels supermachés sont représentés](#stores)
   
   5.5 [Quelles catégories sont représentées](#categories)
   
   5.6 [Conclusion](#conclusion)
6. [Analyse qualitative](#quality)

   6.1 [Détection des features biaisées](#skewed_features)
   
   6.2 [Détection des outliers](#outliers)
   
   6.3 [Définition des données à compléter](#complete)
   
   6.4 [Identification des informations redondantes](#redund)
   
   6.5 [Identification des doublons](#duplicates)
7. [Proposition d'application](#application)

In [ ]:
import numpy as np 
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline

from wordcloud import WordCloud, STOPWORDS

#On ne prend qu'un échantillon du fait des limitations mémoire de la machine
data=pd.read_csv('../dataset/en.openfoodfacts.org.products.csv',sep='\t',nrows=300000, low_memory=False)

<b style="background-color:tomato;font-size:14px;">2. EXAMEN DE FORME<b>
   <a id="exam"></a>  

In [ ]:
shape = data.shape
rows = shape[0]
print(shape)

In [ ]:
list (data.columns)
dataTypeSeries = data.dtypes

In [ ]:
rows = data.shape
rows = rows[0]
data.head()

<b  style="background-color:tomato;font-size:14px;">3. DEFINITION DES MOTS CLE<b>
    <a id="keys"></a> 

<p>Le dataset permet de catégoriser 4 grandes familles de features pour décrire un produit :</p> 
<table>
    <tr>
       <td align="center" bgcolor="red">Description</td>
        <td align="center" bgcolor="red">Type de variable</td>
        <td align="center" bgcolor="red">Exemple</td>
    </tr>    
    <tr>
        <td>Features de référencement produit</td>
        <td>Qualitative</td>
        <td>product_name: nom du produit</td>
    </tr>
    <tr>
        <td>Features de caractérisation du produit sous forme d'informations structurées</td>
        <td>Qualitative</td>
        <td>categories_tags: catégories du produit</td>
    </tr>
    <tr>
        <td>Features de composition du produit (ingrédients, additifs)</td>
        <td>Qualitative</td>
        <td>additives: liste des additifs présents dans le produit</td>
    </tr>
    <tr>
        <td>Features de détail nutritionnel du produit</td>
        <td>Quantitative</td>
        <td>vitamin-d_100g: grammage de vitamine d pour 100g de produit</td>
    </tr>

</table>    


<b style="background-color:tomato;font-size:14px;">3.1 Catégories<b>
    <a id="categ"></a> 

On s'intéresse ici à la feature categories aux catégories de produit définies dans le dataset
#expliquer pourquoi on choisit cette feature : elle permet d'expliquer comment se catégorisent les prd

In [ ]:
def count_words(df, colonne = 'categories_en'):
    list_words = set()
    for word in df[colonne].str.split(','):
        if isinstance(word, float): continue
        list_words = set().union(word, list_words)       
    print("Nb de catégories dans '{}': {}".format(colonne, len(list_words)))
    return list(list_words)
category_keys = count_words(data, 'categories_en')

Illustrons par un nuage de mots (wordcloud) les catégories qui ressortent

In [ ]:
count_keyword = dict()
for index, col in data['categories_en'].iteritems():
    if isinstance(col, float): continue
    for s in col.split(','):
        if s in count_keyword.keys():
            count_keyword[s] += 1
        else:
            count_keyword[s] = 1

keyword_census = []
for k,v in count_keyword.items():
    keyword_census.append([k,v])
keyword_census.sort(key = lambda x:x[1], reverse = True)

In [ ]:
fig = plt.figure(1, figsize=(11,9))
ax1 = fig.add_subplot(1,1,1)
words = dict()
trunc_occurences = keyword_census[0:100]
for s in trunc_occurences:
    words[s[0]] = s[1]

wordcloud = WordCloud(width=900,height=500, background_color='white', 
                      max_words=1628,relative_scaling=0.6,
                      normalize_plurals=False)
wordcloud.generate_from_frequencies(words)
ax1.imshow(wordcloud, interpolation="bilinear")
ax1.axis('off')
plt.show()

En termes chiffrés: 

In [ ]:
keyword_census[:5]

In [ ]:
############## commentaire

<b style="background-color:tomato;font-size:12px;">3.2 Groupes de produits<b>
    <a id="groups"></a> 

In [ ]:
La classification des produits est portée par les deux features pnns_groups_1 et pnns_groups_2
######### dire comment on a compris ça

In [ ]:
pnns_group1_keys = count_words(data, 'pnns_groups_1')  
pnns_group2_keys = count_words(data, 'pnns_groups_2')

In [ ]:
###### etude du contenu en vue de montrer ce que ces catégories signifient et qu'il y a des regroupements possibles

In [ ]:
pnns_group1_keys

In [ ]:
pnns_group2_keys

In [ ]:
######### Créer une fonction qui montre les similitudes et donc les regroupements possibles

<p>Le groupe 2 de PNNS est un détail en sous-groupes du groupe 1 de PNNS. Il apparait plus pertinent de se référer à cette définition de catégorie du produit pour le définir plus précisément.</p>
<p>On constate par ailleurs qu'il y a des regroupements possibles car les libellés sont similaires (anglais vs français, ou bien à une lettre près il s'agit de la même catégorie). On définit une nouvelle feature PNNS_3 qui va effectuer ces regroupements quand ils sont possibles et conserver la définition originale pour les lignes uniques :</p>
<table>
    <tr>
       <td align="center" bgcolor="blue">Libellé de Regroupement</td>
        <td align="center" bgcolor="blue">Catégories regroupées</td>
    </tr>    
    <tr>
        <td>Fruit juices</td>
        <td>Fruit juices,Fruit nectars</td>
    </tr>
    <tr>
        <td>Vegetables</td>
        <td>Legumes,vegetables,Vegetables</td>
    </tr>
    <tr>
        <td>Fruits</td>
        <td>fruits,Fruits</td>
    </tr>
    <tr>
        <td>Sweetened beverages</td>
        <td>Artificially sweetened beverages,Sweetened beverages</td>
    </tr>
    <tr>
        <td>Pizza pies and quiches</td>
        <td>Pizza pies and quiche,Pizza pies and quiches</td>
    </tr>
    <tr>
        <td>Cereals</td>
        <td>cereals,Cereals</td>
    </tr>
    <tr>
        <td>Nuts</td>
        <td>Nuts</td>
    </tr>
    <tr>
        <td>Ice cream</td>
        <td>Ice cream</td>
    </tr>
    <tr>
        <td>Fish and seafood</td>
        <td>Fish and seafood</td>
    </tr>
    <tr>
        <td>Unsweetened beverages</td>
        <td>Unsweetened beverages</td>
    </tr>
    <tr>
        <td>Sweets</td>
        <td>Sweets</td>
    </tr>
    <tr>
        <td>Cheese</td>
        <td>Cheese</td>
    </tr>
    <tr>
        <td>Processed meat</td>
        <td>Processed meat</td>
    </tr>
    <tr>
        <td>Waters and flavored waters</td>
        <td>Waters and flavored waters</td>
    </tr>
    <tr>
        <td>Soups</td>
        <td>Soups</td>
    </tr>
    <tr>
        <td>Dairy desserts</td>
        <td>Dairy desserts</td>
    </tr>
    <tr>
        <td>unknown</td>
        <td>unknown</td>
    </tr>
    <tr>
        <td>Biscuits and cakes</td>
        <td>Biscuits and cakes</td>
    </tr>
    <tr>
        <td>Meat</td>
        <td>Meat</td>
    </tr>
    <tr>
        <td>Bread</td>
        <td>Bread</td>
    </tr>
    <tr>
        <td>Sandwiches</td>
        <td>Sandwiches</td>
    </tr>
    <tr>
        <td>One-dish meals</td>
        <td>One-dish meals</td>
    </tr>
    <tr>
        <td>Fats</td>
        <td>Fats</td>
    </tr>
    <tr>
        <td>Chocolate products</td>
        <td>Chocolate products</td>
    </tr>
    <tr>
        <td>Plant-based milk substitutes</td>
        <td>Plant-based milk substitutes</td>
    </tr>
    <tr>
        <td>Milk and yogurt</td>
        <td>Milk and yogurt</td>
    </tr>
    <tr>
        <td>Dressings and sauces</td>
        <td>Dressings and sauces</td>
    </tr>
    <tr>
        <td>Appetizers</td>
        <td>Appetizers</td>
    </tr>
    <tr>
        <td>Potatoes</td>
        <td>Potatoes</td>
    </tr>
    <tr>
        <td>Alcoholic beverages</td>
        <td>Alcoholic beverages</td>
    </tr>
    <tr>
        <td>Dried fruits</td>
        <td>Dried fruits</td>
    </tr>
    <tr>
        <td>Eggs</td>
        <td>Eggs</td>
    </tr>
    <tr>
        <td>pastries</td>
        <td>pastries</td>
    </tr>
    <tr>
        <td>Teas and herbal teas and coffees</td>
        <td>Teas and herbal teas and coffees</td>
    </tr>
    <tr>
        <td>Breakfast cereals</td>
        <td>Breakfast cereals</td>
    </tr>
    <tr>
        <td>Offals</td>
        <td>Offals</td>
    </tr>
    <tr>
        <td>Salty and fatty products</td>
        <td>Salty and fatty products</td>
    </tr>
</table>

<b style="background-color:tomato;font-size:14px;">4. ANALYSE QUANTITATIVE DES FEATURES</b>
<a id="qanalyse"></a> 

<b style="background-color:tomato;font-size:12px;">6.1 Représentativité des features</b>
<a id="skewed_features"></a> 

In [ ]:
columns = data.columns
over_skewed={}

for c in columns:
    try:
        counts = data[c].value_counts()
        counts = counts.to_frame(name='nb')
        counts['pourcentage'] = counts['nb']/rows
        pct = round(sum(counts[0:10]['pourcentage']),2)
        if pct > 0.05:
            over_skewed[c] = pct
    except:
        i=1
t = dict(sorted(over_skewed.items(), key=lambda item: item[1]))
unusual_df = pd.DataFrame.from_dict(t, orient='index')
unusual_df.plot(kind='bar', figsize=(12,8), title="Représentativité des features dans le dataset", legend=False)

<b style="background-color:tomato;font-size:12px;">6.1 Densité de remplissage des features</b>
<a id="skewed_features"></a> 

In [ ]:
On cherche à montrer comment est rempli le dataset

In [ ]:
percent_of_nans = data.isnull().sum().sort_values(ascending=False) / data.shape[0] * 100

In [ ]:
plt.figure(figsize=(10,5))
sns.displot(percent_of_nans, bins=100, kde=False)
plt.xlabel("% nans")
plt.ylabel("Nombre de features")
plt.title("Dispersion de nans dans les features")

<p>La mise en perspective du nombre de NaN par colonne permet de montrer :</p> 
<p>&nbsp;&nbsp;&nbsp;&nbsp;(1) que la plupart des features ne comporte aucune information (quasi 100% NaN)</p>
<p>&nbsp;&nbsp;&nbsp;&nbsp;(2) qu'il y a une part de features qui ont moins de 20% de NaN</p>
<p>
<p>Pour les features du groupe (1) on peut proposer de ne pas les conserver </p>
<p>Pour les features du groupe (2) il sera intéressant d'appliquer une méthode pour tenter de valoriser les NaN </p>

<b style="background-color:tomato;font-size:12px;">4.1 Full NaN features</b>
<a id="full_nan"></a> 

In [ ]:
useless_features = percent_of_nans[percent_of_nans == 100].index
useless_features

<b style="background-color:tomato;font-size:12px;">4.2 Zero NaN features</b>
<a id="zero_nan"></a> 
<p>Combien de features sont toujours valorisées et quelles sont-elles ?</p>

In [ ]:
zero_nan_features = percent_of_nans[percent_of_nans == 0].index

print('Nombre de features systématiquement définies: ' + str(len(zero_nan_features)))
zero_nan_features

<b style="background-color:tomato;font-size:12px;">4.3 Partiel NaN features</b>
<a id="partial_nan"></a> 
<p>On propose une analyse par quartile pour dégager 3 groupes de features :</p>
<table>
    <tr>
       <td align="center" bgcolor="red">Groupe</td>
        <td align="center" bgcolor="red">Catégorie</td>
        <td align="center" bgcolor="red">What to do</td>
    </tr>    
    <tr>
        <td>0% - 20%</td>
        <td>Features à densité élevée</td>
        <td>Features peu biaisée et donc fiables pour une exploitation dans le cadre d'une application. Pour ces features il serait intéressant de valoriser les NaN par application de la moyenne</td>
    </tr>
    <tr>
        <td>20% - 50%</td>
        <td>Features à densité moyenne</td>
        <td>Features moyennement valorisées, qu'il faudrait soit valoriser pour qu'elles soient pertinentes dans le cadre d'une application (appliquer peut être alors un algorithme knn ?), soit éviter de faire reposer l'analyse dessus.</td>
    </tr>
    <tr>
        <td>50% - 100%</td>
        <td>Features à densité faible</td>
        <td>Ces features sont trop peu valorisées, il ne semble pas judicieux de les utiliser comme fondement pour mener une analyse</td>
    </tr>
    
</table>


In [ ]:
low_nans = percent_of_nans[percent_of_nans <= 20]
middle_nans = percent_of_nans[(percent_of_nans > 20) & (percent_of_nans <= 50)]
high_nans = percent_of_nans[(percent_of_nans > 50) & (percent_of_nans < 100)]

<b>0% - 20% NaN</b>
####################ressortir le nb de cols par tranches

In [ ]:
def rotate_labels(axes):
    for item in axes.get_xticklabels():
        item.set_rotation(60)

In [ ]:
plt.figure(figsize=(20,5))
lows = sns.barplot(x=low_nans.index.values, y=low_nans.values, palette="Greens")
rotate_labels(lows)
plt.title("Features avec peu de NaN (<20%)")
plt.ylabel("% de NaN ")

<b>20% - 50% NaN</b>

In [ ]:
plt.figure(figsize=(20,5))
middle = sns.barplot(x=middle_nans.index.values, y=middle_nans.values, palette="Blues")
rotate_labels(middle)
plt.title("Features avec un nombre moyen de NaN")
plt.ylabel("% de NaN ")

In [ ]:
Il est intéressant de remarquer que dans cette catégorie on retrouve un certain nombre de features "tags", qui sont porteuses de détails intéressants :

In [ ]:
data.loc[11000,['additives_tags', 'additives_en', 'additives_n']]

<b>50% - 100% NaN</b>

In [ ]:
plt.figure(figsize=(15,30))
high = sns.barplot(y=high_nans.index.values, x=high_nans.values, palette="Reds")
plt.title("Features comportant une grande part de NaN (> 50%)")
plt.ylabel("% de NaN ")

<p>A première vue la grande partie des features entrant dans ce cas sont les détails fins d'analyse nutritionnelle (vitamines, oligo-éléments, ...). Cela peut se comprendre si l'utilisateur qui a effectué l'enregistrement soit n'en a pas connaissance car cela n'est pas répertorié sur l'étiquette du produit, soit parce que l'étape de saisie est trop fastidieuse. </p> 
</p>On constate pour ce groupe que le % de NaN est au minimum de 75%. Cela rend l'exploitation de ces features quasi impossible. Je pense que nous pouvons raisonnablement écarter ces features.<p>

<br>
<p>Parmi les éléments remarquables, on peut noter que les features suivantes ne sont jamais renseignées :</p>
<p>&nbsp;&nbsp;&nbsp;&nbsp;origine<p>
<p>&nbsp;&nbsp;&nbsp;&nbsp;lieu de fabrication<p>    
<p>Là également on peut comprendre que cette information ne soit pas bien valorisée, mais elle met en lumière la problématique de traçabilité d'un produit.<p>
   

<b style="background-color:tomato;font-size:14px;">4. ANALYSE DE CORRELATION DES FEATURES</b>
<a id="qanalyse"></a> 

On cherche à détecter les relations linéaires qui existent éventuellement entre les variables du dataset. 
On retient pour cela la représentation de la matrice de correlation du coefficient de relation linéaire de Pearson.
L'objectif dans un premier temps est d'identifier les correlations très fortes qui indiqueraient les features porteuses d'information similaires et dont on pourrait alors ne retenir qu'un représentant.

In [ ]:
sns.set(context="paper", font_scale = 1.2)
corrmat = data.corr()
f, ax = plt.subplots(figsize=(12, 12))
f.text(0.45, 0.93, "Matrice des coefficients correlation de Pearson", ha='center', fontsize = 18)
sns.heatmap(corrmat, square=True, linewidths=0.01, cmap="coolwarm")
plt.tight_layout()

On remarque: 
    - des features -xxxx_100g : elles correspondent au détail fin de la famille de nutriment. En fonction de notre application si on prend bien en considération la famille de nutriment on peut alors écarter ces features
    

    - des features libelle, libelle_tags,libelle_en: ces features correspondent à une déclinaison similaire d'une même information. On peut proposer de n'en retenir qu'une
    #########montrer un exemple ?

In [ ]:
list_columns = ['categories', 'categories_tags', 'categories_en']
data[data[list_columns].notnull().any(axis=1)][['product_name']+ list_columns][:20:3]

Démontrer que les infos sont similaires en mettant en valeur que le contenu d'une feature se retrouve dans d'autres features identifiées comme redondantes (graphs)

<p>Dans la phase de cleaning on ne conservera que les features '_tags' pour permettre une exploitation des informations (notamment l'association langue:terme)</p>

<b style="background-color:tomato;font-size:14px;">5. AUDIT DU DATASET</b>
<a id="actors"></a> 

<p>Au point précédent nous avons constaté que les informations nutritionnelles fines n'étaient quasi pas renseignées. Cela nous amène à vérifier comment sont valorisées les informations du dataset (institutions, pays, etc ...)</p>

In [ ]:
<b style="background-color:tomato;font-size:12px;">5.1 Qui sont les créateurs d'information</b>
<a id="creators"></a> 

In [ ]:
def top_n(col,col_alias,n):
    counts = data[col].value_counts()
    counts = counts.to_frame(name='nombre')
    counts['pourcentage'] = (counts['nombre']/rows)*100

    top_n = round(sum(counts[0:n]['pourcentage']),2)*100
    print("Le top " + str(n) + " des", col_alias,"cumule", str(top_n/100),"% du périmètre : \n")

    # Pie chart
    sub_df=counts.head(n)
    labels = sub_df.iloc[ : , 0 ]
    labels=labels.index
    sizes = sub_df.iloc[ : , 1 ]
    explode = tuple()
    for i in range(n):
        explode += (0.1, )

    # Creating color parameters 
    colors = ( "orange", "cyan", "brown", 
              "grey", "indigo", "beige") 

    # Wedge properties 
    wp = { 'linewidth' : 1, 'edgecolor' : "green" } 

    # Creating autocpt arguments 
    def func(pct, allvalues): 
        absolute = int(pct / 100.*np.sum(allvalues)) 
        return "{:.1f}%".format(pct, absolute) 

    # Creating plot 
    fig, ax = plt.subplots(figsize =(10, 7)) 
    wedges, texts, autotexts = ax.pie(sizes,  
                                      autopct = lambda pct: func(pct, sizes), 
                                      explode = explode,  
                                      labels = labels, 
                                      shadow = True, 
                                      colors = colors, 
                                      startangle = 90, 
                                      wedgeprops = wp, 
                                      textprops = dict(color ="magenta")) 

    # Legend 
    ax.legend(wedges, labels, 
              title ="Contributeurs", 
              loc ="center left", 
              bbox_to_anchor =(1, 0, 0.5, 1)) 

    plt.setp(autotexts, size = 8, weight ="bold") 

    plt.show()     

In [ ]:
top_n('creator','créateurs',5)

Le principal contributeur est le département de l'agriculture américain (https://www.usda.gov/)

<b style="background-color:tomato;font-size:12px;">5.2 Quels sont les pays représentés</b>
<a id="countries"></a> 

In [ ]:
top_n('countries','pays',5)

<p>On retrouve principalement les Etats Unis, puis la France.</p>
<p>Dans l'idée de l'application de santé publique du ministère français il me semble approprié de ne pas conserver les données US qui présentent un biais trop important, d'autant que les produits US ne sont pas nécessairement disponibles en France.<p>

<b style="background-color:tomato;font-size:12px;">5.3 Quelles marques sont représentées</b>
<a id="brands"></a> 

In [ ]:
top_n('brands','marques',10)

Top 10 des marques en France

In [ ]:
france = data[(data['countries']=="France") | (data['countries']=="en:FR") | (data['countries']=="en:france")]
y=france['brands'].value_counts().head(10)
x=france['brands'].value_counts().head(10).index
plt.figure(figsize=(20,5))
show = sns.barplot(x=x, y=y,hue=y, palette=("Oranges"))
rotate_labels(show)
show.legend_.remove()
plt.title("top 10 des marques en France")

<b style="background-color:tomato;font-size:12px;">5.4 Quels supermachés sont représentés</b>
<a id="stores"></a> 

In [ ]:
top_n('stores','Supermarchés tous pays confondus',10)

Top 10 des supermarchés en France

In [ ]:
france = data[(data['countries']=="France") | (data['countries']=="en:FR") | (data['countries']=="en:france")]
y=france['stores'].value_counts().head(10)
x=france['stores'].value_counts().head(10).index
plt.figure(figsize=(20,5))
show = sns.barplot(x=x, y=y,hue=y, palette=("Oranges"))
rotate_labels(show)
show.legend_.remove()
plt.title("top 10 des supermarchés en France")

<b style="background-color:tomato;font-size:12px;">5.5 Quelles catégories sont représentées</b>
<a id="categories"></a> 

In [ ]:
fig = plt.figure(figsize=(12,6))
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)

fr_values = france['main_category'].value_counts()
us_values = us['main_category'].value_counts()
fr_values[0:10].plot(kind='bar', ax=ax1, title="France")
us_values[0:10].plot(kind='bar', ax=ax2, title="US")
print('                       Quelles sont les catégories les plus représentées ?')
plt.show()

In [ ]:
def rotate_labels(axes):
    for item in axes.get_xticklabels():
        item.set_rotation(60)

In [ ]:
def univariateBarPlotting(var, title):
    plt.figure(figsize=(20,5))
    fig = sns.barplot(y=data[var].index.values, x=data[var].values, color="blue")
    rotate_labels(fig)
    plt.title(title)

In [ ]:
univariateBarPlotting('pnns_groups_1','Quantité de produits par groupe')

In [ ]:
data['pnns_groups_1'].describe()

In [ ]:
univariateBarPlotting('pnns_groups_2','Quantité de produits par sous-groupe')

In [ ]:
def showSubCategContents(v):
    
    subset=data[(data['pnns_groups_2']==v)]
    ar=subset.values
    ar=ar[:30,7]

    fig = plt.figure()
    fig.suptitle('Zoom sur les 30 premiers aliments de la sous groupe ' + v, fontsize=12, fontweight='bold')

    ax = fig.add_subplot(111)

    i=0
    k=0
    for wd in enumerate(ar):
        if i%2 == 0:
            ax.text(1, (i+1), wd[1], color='green', fontsize=8)
        else:     
            ax.text(3, (i+1), wd[1], color='blue', fontsize=8)
        i = i + 1
    ax.axis([3, 0, 30, 0])   
    plt.axis('off')
    plt.show()

In [ ]:
showSubCategContents("Meat")

In [ ]:
showSubCategContents("Cereals")

In [ ]:
showSubCategContents("unknown")

<b style="background-color:tomato;font-size:12px;">5.6 Conclusion</b>
<a id="conclusion"></a> 

On constate qu'à première vue le dataset comporte de nombreux produits issus de la grande distribution. 
Sans préjuger de leur qualité nutritionnelle, il semble intéressant dans la cadre de la conception d'une application en lien avec la nutrition de mettre en perspective la qualité nutritionnelle de ces produits avec des produits bruts.
D'autre part dans les groupes (pnns_groups_1) et sous groupe (pnns_groups_2) les entrées correspondant à "unknown", qui représentent (/!\ donner des chiffres) ne présentent pas d'intérêt à être conservées, elles seront purgées dans la phase de clean. 

<b style="background-color:tomato;font-size:14px;">6. ANALYSE QUALITATIVE</b>
<a id="quality"></a> 

On vérifie la présence d'outliers dans les différentes features porteuses d'information nutritionnelle. L'idée est d'évaluer la cohérence portée par celles-ci et d'avoir une idée du travail d'ajustement nécessaire.
Pour établir une règle générale on procède à un échantillonage sur le périmètre de ces features. On défira alors le seuil le plus approprié. 

In [ ]:
quantite = ['energy_100g', 'vitamin-a_100g', 'vitamin-c_100g', 'vitamin-pp_100g',
            'vitamin-b6_100g','vitamin-b9_100g','pantothenic-acid_100g', 'biotin_100g',
            'salt_100g','fat_100g','fiber_100g','sugars_100g']

In [ ]:
sigma = [0 for _ in range(len(quantite))]
mediane = [0 for _ in range(len(quantite))]
for i in range(len(quantite)):
    colonne = quantite[i]
    mediane[i] = data[pd.notnull(data[colonne])][colonne].median()
    test = data[pd.notnull(data[colonne])][colonne]
    test = test.sort_values()    
    sigma[i] = np.std(test[:-25])    

In [ ]:
tPlot, axes = plt.subplots(nrows=4, ncols=3, sharex=False, sharey=False, figsize=(11,11))
axes = np.array(axes)
sigma_factor = 10
i=0
for ax in axes.reshape(-1):
    colonne = quantite[i]
    test = data[pd.notnull(data[colonne])][colonne]
    ax.tick_params(labelcolor='black',top='off',bottom='on',left='on',right='off',labelsize=6)
    ax.set_ylabel(colonne.rstrip("_100g"), fontsize = 10)
    ax.set_yscale("log")
    ax.plot(list(test), 'b.', markeredgewidth = 0.3, markeredgecolor='w')
    for tick in ax.get_xticklabels():
        tick.set_rotation(30)
    ax.axhline(y=mediane[i] + sigma_factor*sigma[i], color='r', linestyle='-')
    
    ax.text(0., 0.02, ' median:{:.3} \n sigma:{:.3}'.format(mediane[i], sigma[i]),
            style='italic', transform=ax.transAxes, fontsize = 10,
            bbox={'facecolor':'green', 'alpha':0.5, 'pad':0})
    i += 1

tPlot.text(0.5, 1.01, "Mise en évidence des outliers par rapport au seuil " + str(sigma_factor) + " * sigma", ha='center', fontsize = 12)
plt.tight_layout()

Après plusieurs ajustements du facteur de déviation standard des valeures de feature par rapport à la médiane on arrive à un bon compromis avec 10 sigma. 
On va donc retenir cette valeur pour écarter les outliers dans nos traitements de cleaning en définissant que si une valeur de feature dépasse de 10 sigma la valeur médiane de cette feature on l'écarte du dataset, à la condition que la valeur apparemment en outling ne soit pas portée par un seul ingrédent (comme par exemple le sel).  

<b style="background-color:tomato;font-size:12px;">6.3 Définition des données à compléter<b>
    <a id="complete"></a> 

Dans l'optique d'une application de santé publique on s'appuyera nécessairement sur les qualités intrinsèques du produit. Ces qualités sont portées par les features quantitatives. On propose donc de partir du périmètre des features quantitatives dont on va analyser la densité de contenu. Pour celles dont la densité est comprise dans la fourchette de 75% à 100% on proposera de les valoriser par une méthode de régression (point développé dans le notebook de clean).  

In [ ]:
to_be_completed = percent_of_nans[(percent_of_nans < 25) & (percent_of_nans > 0)]
to_be_completed_high_density = to_be_completed.filter(like='_100g')
to_be_completed_high_density

Pour celles dont la densité est comprise dans la fourchette de 50% à 75% on proposera de les valoriser par application de la valeur médiane.

In [ ]:
to_be_completed = percent_of_nans[(percent_of_nans < 50) & (percent_of_nans >= 25 )]
to_be_completed_average_density = to_be_completed.filter(like='_100g')
to_be_completed_average_density

<b style="background-color:tomato;font-size:12px;">6.4 Identification des informations redondantes<b>
    <a id="redund"></a> 

On remarque à première vue que la liste de features comporte des noms redondants.
Ex: categories, categories_en, categories_tag

In [ ]:
<b style="background-color:tomato;font-size:12px;">6.5 Identification des doublons<b>
    <a id="duplicates"></a> 

In [ ]:
duplicates = []
for i,dup in enumerate(data.duplicated(subset=(['product_name','creator']))):
    if dup: 
        s = data["product_name"][i]
        if not pd.isna(s):
            r = [s,data["creator"][i]]
            duplicates.append(r)
len(duplicates)            

In [ ]:
duplicates[0:10]

In [ ]:
data.loc[(data['product_name'] == "Miel") & (data['creator'] == "kiliweb")]

Sur cet exemple on comprend que pour un produit donné et un contributeur donné on a plusieurs entrées sans pouvoir définir laquelle pourrait être gardée. En effet il n'y a pas de critère évident qui permettrait d'affiner la sélection pour ne retenir que le plus pertinent (comme la date de dernière modification qui pourrait signifier que les données de la ligne ont été mises à jour avec de meilleures données).
En l'état nous ne projetons pas de traiter les doublons.
####### démontrer que ce ne sont pas des doublons, concaténer les valeurs de features et ressortir si doublons ou pas)


In [ ]:
del data
del percent_of_nans
del unusual_df
del to_be_completed
del to_be_completed_high_density
del to_be_completed_average_density

<b style="background-color:tomato;font-size:14px;">7. PROPOSITION D'APPLICATION<b>
    <a id="application"></a> 


<p>Les objectifs affichés du Ministère de Santé Publique en matière de pratiques alimentaires sont clairement définis dans la rubrique Recommandations de son site lié à la nutrition : <a href="https://www.mangerbouger.fr/">site mangerbouger</a></p>
<p>Ces recommandations ciblent des catégories d'aliments pour lesquelles le Ministère définit une tendance à les intégrer dans les repas (Augmenter, Aller vers, Réduire)</p>

<p>De façon à quantifier l'impact de ces recommandations il est important de s'appuyer sur des indicateurs clairement établis. On distingue 2 indicateurs :</p>
<p>- le score nutritionnel : il permet, sur la base de la composition du produit (aliment brut ou bien produit transformé) en différents nutriments et ingrédients majeurs pour la santé, de donner une valeur unique d’estimation de la qualité nutritionnelle de l’aliment, sur une échelle ordinale continue allant de -15
(meilleure qualité nutritionnelle) à +40 (pire qualité nutritionnelle) </p>
<p>- le nutriscore : il définit une échelle graphique qui scinde le score nutritionnel en 5 classes (exprimées par une couleur associée à une lettre) et vise à faciliter la visibilité, la lisibilité, et la compréhension de la qualité nutritionnelle par le consommateur</p>

<p>voir références:</p>
<p>&nbsp;&nbsp;&nbsp;-&nbsp;<a href="https://fr.wikipedia.org/wiki/Nutri-score#M%C3%A9thode_de_calcul_du_score">Définition Wikipedia</a></p>
<p>&nbsp;&nbsp;&nbsp;-&nbsp;<a href="https://www.santepubliquefrance.fr/content/download/150262/file/QR%20scientifique%20et%20technique_271020.pdf">Définition Ministère santé publique</a></p>
<p>D'autre part nous avons pu constater que les acteurs et les produits étaient fortement liés à la grande distribution. Il serait intéressant de mettre en perspective la qualité nutritionnelle découlant de la mise en oeuvre de ces produits dans le cadre d'un repas par rapport aux recommandations du Ministère de la Santé.</p>
<p>Pour cela on pourrait concevoir une application qui permettrait d'illuster les bienfaits de l'application de ces recommandations.</p>
<p>L'application devrait montrer l'impact sur le score nutritionnel global d'un repas quand une part des ingrédients est modifiée en fonction de ces recommandations.</p>
<p>Pour cadrer les features qui devront être particulièrement prises en compte dans l'établissement du scoring nous reprenons ci-dessous les informations déduites des préconisations du Ministère:</p>
<p>
<table>
    <tr>
        <td align="center" bgcolor="green">Feature</td>
        <td align="center" bgcolor="green">Impact</td>
    </tr>
    <tr>
        <td>Apport calorique</td>
        <td>Négatif</td>
    </tr>
    <tr>
        <td>Sucre</td>
        <td>Négatif</td>
    </tr>
    <tr>
        <td>Graisses saturées</td>
        <td>Négatif</td>
    </tr>
    <tr>
        <td>Sel</td>
        <td>Négatif</td>
    </tr>
    <tr>
        <td>Fruits, légumes, légumineuses, fruits à coque et huiles de colza, de noix et d’olive</td>
        <td>Positif</td>
    </tr>
    <tr>
        <td>protéines</td>
        <td>Positif</td>
    </tr>
    <tr>
        <td>Fibres</td>
        <td>Positif</td>
    </tr>
</table>
</p>
<p>A noter deux remarques importantes : </p>
<p>&nbsp;&nbsp;&nbsp;&nbsp; - les fromages, bien qu'ils contiennent des graisses saturées et sont caloriques et salés, ne sont pas à écarter du fait de leur proportion importante de protéines et de calcium</p>
<p>&nbsp;&nbsp;&nbsp;&nbsp; - la teneur en fruit et légumes ne doit pas tenir compte des féculents</p>

Fonctionnellement, l'application consiste à permettre la saisie des informations relatives au repas évalué (apéro, entrée, plat, fromage/laitage, dessert) et vise à détecter des composantes de faible score nutritionnel et pour ces composantes à indiquer des alternatives qui offriraient un meilleur score nutritionnel global. L'application peut mettre en vis-à-vis un graphique radar du repas et de celui corrigé par ces alternatives. L'idée de l'application est de mettre en valeur les alternatives pour que l'utilisateur apprenne et puisse au fur et à mesure faire évoluer ses habitudes. Ayant une base de connaissance, lors de ses futurs achats en vue de préparer un repas il pourra alors mieux se diriger vers ces alternatives.